In [1]:
!pip install --upgrade pip
!pip install transformers==4.20.0
!pip install datasets
!pip install huggingface-hub
!pip install nltk
!pip install rouge-score
!pip install keras-nlp==0.3.1
!pip install git+https://github.com/google-research/bleurt.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-qdq2124o
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-q

In [2]:
import nltk
import numpy as np
np.random.seed(26)
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
MAX_INPUT_LENGTH = 1024  # Maximum length of the input to the model
MIN_TARGET_LENGTH = 5  # Minimum length of the output by the model
MAX_TARGET_LENGTH = 512  # Maximum length of the output by the model
BATCH_SIZE = 8  # Batch-size for training our model
LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for
N_ROWS = 1000

# This notebook is built on the t5-small checkpoint from the Hugging Face Model Hub
MODEL_CHECKPOINT = "t5-small"

In [ ]:
train = pd.read_csv("cnn_dailymail/train.csv")
test = pd.read_csv("cnn_dailymail/test.csv")
val = pd.read_csv("cnn_dailymail/validation.csv")

sub_train = train.sample(N_ROWS*3)
sub_train.to_csv("cnn_dailymail/sub_train.csv", index=False)

sub_test = test.sample(N_ROWS)
sub_test.to_csv("cnn_dailymail/sub_test.csv", index=False)

sub_val = test.sample(N_ROWS)
sub_val.to_csv("cnn_dailymail/sub_validation.csv", index=False)

del train, test, val, sub_train, sub_test, sub_val

In [ ]:
data_files = {"train": "sub_train.csv", "test": "sub_test.csv", "validation": "sub_validation.csv"}
dataset = load_dataset("cnn_dailymail/", data_files=data_files)
dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [ ]:
prefix = "summarize: "

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=MAX_TARGET_LENGTH, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_datasets

In [ ]:
batch_size = 8
model_name = "t5-small-summarization-fine-tuned"
model_dir = f"Models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=40,
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="steps",
    save_steps=40,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1"
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)
bleurt = load_metric("bleurt")
rouge = load_metric("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Compute ROUGE scores
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    bleurt_result = bleurt.compute(predictions=decoded_preds, references=decoded_labels)
 
    # Extract ROUGE f1 scores
    rouge_result = {key: value.mid.fmeasure * 100 for key, value in rouge_result.items()}
    bleurt_score = np.mean(bleurt_result["scores"])
    
    result = rouge_result
    result["bleurt"] = bleurt_score

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
!zip -r /content/models.zip /content/Models